In [2]:
from bs4 import BeautifulSoup as BS
from tqdm import tqdm
import re
import json
import pandas as pd
import glob
from spacy.matcher import PhraseMatcher
from spacy.lang.en import English
import spacy

from nltk import sent_tokenize

In [3]:
nlp = English()
matcher = PhraseMatcher(nlp.vocab)

In [50]:
filepath = './data/ctrn-20220129x10k.html'

with open(filepath, encoding='utf-8') as fp:
    test_file = BS(fp, 'html.parser')

#print(test_file.prettify())

test_file=test_file.text

sentences_tokenized=sent_tokenize(test_file)
#sentences_tokenized    

In [64]:
sentences=[]

for sentence in sentences_tokenized:
    match=re.search(r"(share|stock) repurchase (program)?", sentence)
    if match: 
        match = re.search(r'authorized|approved', sentence)
        if match:
            sentences.append(sentence)  
    

In [65]:
sentences

['(2)  On November 30, 2021, the Company announced that its board of directors approved a $30 million stock repurchase program.',
 'On March 15, 2022, the Company announced that its board of directors approved an additional $30 million stock repurchase program.',
 'In March 2022, the Company announced that its board of directors approved an additional $30.0 million stock repurchase program.\u200bStock-Based CompensationThe Company maintains the Citi Trends, Inc. Incentive Plan (the “Plan”) which permits the grant of stock-based incentive awards to employees, officers, directors and consultants.']

In [61]:
paragraph = [re.sub(r"\\[a-z]+\d?", " ", repr(sentence).strip("'")) for sentence in sentences]

In [62]:
paragraph

['(2)  On November 30, 2021, the Company announced that its board of directors approved a $30 million stock repurchase program.',
 'On March 15, 2022, the Company announced that its board of directors approved an additional $30 million stock repurchase program.',
 'In March 2022, the Company announced that its board of directors approved an additional $30.0 million stock repurchase program. 00bStock-Based CompensationThe Company maintains the Citi Trends, Inc. Incentive Plan (the “Plan”) which permits the grant of stock-based incentive awards to employees, officers, directors and consultants.']

another way to remove html part
for script in test_file(["script", "style"]):
    script.decompose()


script = list(test_file.stripped_strings)
print(script)

In [73]:
nlp = spacy.load('en_core_web_sm')

In [70]:
for p in paragraph:
    doc=nlp(p)
    for ent in doc.ents:
        if ent.label_ in ["DATE", "CARDINAL", "MONEY"]:
            print(ent.text, ent.label_)

2 CARDINAL
November 30, 2021 DATE
$30 million MONEY
March 15, 2022 DATE
an additional $30 million MONEY
March 2022 DATE
an additional $30.0 million MONEY


In [13]:
#the code in this cell and the following are from stack overflow to see how the code works 
color_patterns = [nlp(text) for text in ('red', 'green', 'yellow')]
product_patterns = [nlp(text) for text in ('boots', 'coats', 'bag')]
material_patterns = [nlp(text) for text in ('silk', 'yellow fabric')]

matcher = PhraseMatcher(nlp.vocab)
matcher.add('COLOR', None, *color_patterns)
matcher.add('PRODUCT', None, *product_patterns)
matcher.add('MATERIAL', None, *material_patterns)

In [14]:
doc = nlp("yellow fabric")
matches = matcher(doc)
for match_id, start, end in matches:
    rule_id = nlp.vocab.strings[match_id]  # get the unicode ID, i.e. 'COLOR'
    span = doc[start : end]  # get the matched slice of the doc
    print(rule_id, span.text)

COLOR yellow
MATERIAL yellow fabric
